# Дан набор текстовых документов, необходимо найти наиболее частотные слова среди всех документов (>2 повторений, не являются словоформами друг друга), исключить из этого списка служебные части речи и посчитать количество их встречающихся словоформ во всех документах.

In [ ]:
pip install pymorphy3 nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 49.6 MB/s eta 0:00:00


In [ ]:
from collections import defaultdict
import pymorphy3
from nltk.corpus import stopwords
import nltk


In [ ]:
documents = {1: 'В случаях, предусмотренных законом, права, закрепляющие принадлежность объекта гражданских прав определенному лицу, ограничения таких прав и обременения имущества (права на имущество) подлежат государственной регистрации. Государственная регистрация прав на имущество осуществляется уполномоченным в соответствии с законом органом на основе принципов проверки законности оснований регистрации, публичности и достоверности государственного реестра. В государственном реестре должны быть указаны данные, позволяющие определенно установить объект, на который устанавливается право, управомоченное лицо, содержание права, основание его возникновения. Права на имущество, подлежащие государственной регистрации, возникают, изменяются и прекращаются с момента внесения соответствующей записи в государственный реестр, если иное не установлено законом. В случаях, предусмотренных законом или соглашением сторон, сделка, влекущая возникновение, изменение или прекращение прав на имущество, которые подлежат государственной регистрации, должна быть нотариально удостоверена. Запись в государственный реестр вносится при наличии заявлений об этом всех лиц, совершивших сделку, если иное не установлено законом. Если сделка совершена в нотариальной форме, запись в государственный реестр может быть внесена по заявлению любой стороны сделки, в том числе через нотариуса.',

2: 'Гражданин отвечает по своим обязательствам всем принадлежащим ему имуществом, за исключением имущества, на которое в соответствии с законом не может быть обращено взыскание. Перечень имущества граждан, на которое не может быть обращено взыскание, устанавливается гражданским процессуальным законодательством. В п. 55 указанного Постановления Пленума Верховного Суда РФ разъясняется, что при обращении взыскания на имущество должника - индивидуального предпринимателя по требованиям исполнительного документа, не связанным с осуществлением предпринимательской деятельности, применяются правила очередности, предусмотренные статьей 69 Закона об исполнительном производстве. Вместе с тем при обращении судебным приставом-исполнителем взыскания на имущество должника - индивидуального предпринимателя по требованиям, связанным с его предпринимательской деятельностью (пункт 3 статьи 23 ГК РФ), необходимо соблюдать не только очередность, установленную статьей 69 Закона об исполнительном производстве, но и иные положения законов, определяющих очередность взыскания с учетом такого статуса должника, в частности нормы статьи 94 названного Закона.',

3: 'Составной частью особого правового режима недвижимого имущества является обязательная государственная регистрация прав на него (перехода прав на него, сделок с ним и обременений недвижимого имущества). Целью такой регистрации является установление прочной системы оборота недвижимости. Мировой опыт убедительно показывает, что для защиты прав частных собственников на недвижимость необходимо создать формальную регистрационную систему, которая гарантировала бы надежность и гласность гражданского оборота, была бы доступна для всех участников гражданских отношений и содержала бы объективную информацию о недвижимости. Государственная регистрация прав на недвижимое имущество и сделок с ним представляет собой элемент публично-правового регулирования в частноправовых отношениях. Однако первое выполняет вторичную роль по отношению к последним. Цель такого регулирования заключается в том, чтобы обеспечить стабильность в гражданских отношениях. Требование об отражении изменений вещно-правового положения участников в Едином государственном реестре прав на недвижимое имущество является реализацией принципа публичности в обороте недвижимости. Слово "государственная" означает, что регистрация должна осуществляться специально уполномоченными федеральными государственными органами. Таким органом в настоящее время является Федеральная служба государственной регистрации, кадастра и картографии (Росреестр).'}


In [ ]:

nltk.download('stopwords')


morph = pymorphy3.MorphAnalyzer()
russian_stopwords = stopwords.words('russian')


def preprocess_text(text):
    words = text.lower().split()
    normalized_words = []
    for word in words:
        # Удаляем пунктуацию и приводим к нормальной форме
        parsed = morph.parse(word)[0]
        if parsed.tag.POS not in ['PREP', 'CONJ', 'PRCL', 'INTJ']:  # Исключаем служебные части речи
            normalized_word = parsed.normal_form
            if normalized_word not in russian_stopwords and len(normalized_word) > 2:
                normalized_words.append(normalized_word)
    return normalized_words

# Сбор всех слов и их словоформ
word_forms = defaultdict(list)
word_counts = defaultdict(int)

for doc in documents:
    words = preprocess_text(documents[doc])
    for word in words:
        word_counts[word] += 1
        word_forms[word].append(word)

# Фильтрация: оставляем слова с >2 повторениями
filtered_words = {word: count for word, count in word_counts.items() if count > 2}

# Подсчёт словоформ для каждого частотного слова
result = {}
for word in filtered_words:
    forms = set(word_forms[word])  # Уникальные словоформы
    result[word] = {
        'total_occurrences': word_counts[word],
        'unique_forms': len(forms),
        'forms': list(forms)
    }

print(result)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


{'гражданский': {'total_occurrences': 5, 'unique_forms': 1, 'forms': ['гражданский']}, 'право': {'total_occurrences': 10, 'unique_forms': 1, 'forms': ['право']}, 'имущество': {'total_occurrences': 8, 'unique_forms': 1, 'forms': ['имущество']}, 'государственный': {'total_occurrences': 14, 'unique_forms': 1, 'forms': ['государственный']}, 'регистрация': {'total_occurrences': 5, 'unique_forms': 1, 'forms': ['регистрация']}, 'закон': {'total_occurrences': 5, 'unique_forms': 1, 'forms': ['закон']}, 'регистрации,': {'total_occurrences': 4, 'unique_forms': 1, 'forms': ['регистрации,']}, 'реестр': {'total_occurrences': 4, 'unique_forms': 1, 'forms': ['реестр']}, 'должный': {'total_occurrences': 3, 'unique_forms': 1, 'forms': ['должный']}, 'установить': {'total_occurrences': 3, 'unique_forms': 1, 'forms': ['установить']}, 'который': {'total_occurrences': 5, 'unique_forms': 1, 'forms': ['который']}, 'запись': {'total_occurrences': 3, 'unique_forms': 1, 'forms': ['запись']}, 'иной': {'total_occur

In [ ]:
tot = 0
for value in result.values():
  tot += value['total_occurrences']
print(tot)


102
